In [1]:
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()

import pandas as pd

In [3]:
from utils import filter_by_media
from utils import cluster_by_month
#from utils import preprocess

df = pd.read_csv("../../data/loslagos-comunas.csv")
df = cluster_by_month(filter_by_media(df))
df = df.drop_duplicates(subset='content', keep="first")

## 1. Modelado de tópicos con BERTopic

In [4]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from umap import UMAP
from hdbscan import HDBSCAN

nlp = spacy.load("es_core_news_md", exclude=['tagger', 'parser', 'ner', 
                                             'attribute_ruler', 'lemmatizer'])

In [5]:
df['topic'] = ""
docs = df.content.tolist()
df.head()

,date,media_outlet,url,title,text,content,comuna,date_clustering,topic
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...,reconocen guardaparques región lagos actores c...,"['puyehue', 'chaiten']",2021-10,
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...,nuevos materiales comienza plan piloto saltos ...,['puerto varas'],2021-10,
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...,centro salud familiar cesfam puerto varas invi...,['puerto varas'],2021-10,
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...,alcalde tomás gárate presidió primera vez octa...,"['castro', 'puerto varas']",2021-10,
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",galería arte machacoya realizará remate obras ...,NaN,2021-10,


## UMAP y HDBSCAN

[UMAP](https://pair-code.github.io/understanding-umap/) es un algoritmo que sirve para reducir la dimensionalidad de los datos y, en concjunto con HDBSCAN, ayuda en la generación de clusters.

Ambos algoritmos tienen un gran número de parámetros. Sin embargo, los más notables son los  siguientes:

|Algoritmo|Parametro|Descripción|
|---|----|----|
|UMAP|n_neighbors|is the numer of neighboring sample points used when making the manifold approximation. Increasing this value typically results in a more global view of the embedding structure whilst smaller values result in a more local view. Increasing this value often results in larger clusters being created.|
|UMAP|n_components|refers to the dimensionality of the embeddings after reducing them. This is set as a default to 5 in order to reduce dimensionality as much as possible whilst trying to maximize the information kept in the resulting embeddings. Although lowering or increasing this value has an influence on the quality of embeddings, its effect is largest on the performance of HDBSCAN. Increasing this value too much and HDBSCAN will have a hard time clustering the high-dimensional embeddings. Lower this value too much and too little information in the resulting embeddings is available to create proper clusters. If you want to increase this value, I would advise setting using a metric for HDBSCAN that works well in high dimensional data.|
|UMAP|metric|refers to the method used to compute the distances in high dimensional space. The default is cosine as we are dealing with high dimensional data. However, BERTopic is also able to use any input, even regular tabular data, to cluster the documents. Thus, you might want to change the metric to something that fits with your use case.|
|UMAP|low_memory|is used when datasets may consume a lot of memory. Using millions of documents can lead to memory issues and setting this value to True might alleviate some of the issues.|
|HDBSCAN|min_cluster_size|is arguably the most important parameter in HDBSCAN. It controls the minimum size of a cluster and thereby the amount of clusters that will be generated. It is set to 10 as a default. Increasing this value results in less clusters but of larger size whereas decreasing this value results in more micro clusters being generated. Typically, I would advise on increasing this value rather than decreasing it.|
|HDBSCAN|min_samples| is automatically set to min_cluster_size and controls the amount of outliers are generated. Setting this value significantly lower than min_cluster_size might help you reduce the amount of noise you will get. Do note that outliers are typically to be expected and forcing the output to have no outliers may not properly represent the data.|
|HDBSCAN|metric|like with HDBSCAN is used to calculate the distances. Here, we went with euclidean as, after reducing the dimensionality, we have low dimensional data and not much optimization is necessary. However, if you increase n_components in UMAP, then it would be advised to look into metrics that work with high dimensional data.|
|HDBSCAN|prediction_data|Make sure you always set this value to True as it is needed to predict new points later on. You can set this to False if you do not wish to predict any unseen datapoints.|


In [17]:
neighbors=[10, 15, 30, 50]
components=[4,5,6]

cluster_size=[8,10,15]

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=None)

umap_model = UMAP(n_neighbors=15, n_components=5, 
                  metric='cosine', random_state=42) # random_state = 42 porque queremos resultados determinísticos

hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', 
                        prediction_data=True, min_samples=5)

In [20]:
for n in neighbors:
    for c in components:
        for cs in cluster_size:
            print("n_neighbors: ",n)
            print("n_components: ",c)
            print("min_cluster_size: ",cs)

            umap_model = UMAP(n_neighbors=n, n_components=c, 
                                  metric='cosine', random_state=42) 
                
            hdbscan_model = HDBSCAN(min_cluster_size=cs, metric='euclidean', 
                                        prediction_data=True, min_samples=5)

            topic_model = BERTopic(embedding_model=nlp,
                                        vectorizer_model=vectorizer_model,
                                        umap_model=umap_model,
                                        hdbscan_model=hdbscan_model,
                                        #calculate_probabilities=True,verbose=True
                                  )

            topics, probs = topic_model.fit_transform(docs)
                
            print(topic_model.get_topic_info())

n_neighbors:  10
n_components:  4
min_cluster_size:  8


2022-11-03 18:05:51,700 - BERTopic - Transformed documents to Embeddings
2022-11-03 18:06:27,584 - BERTopic - Reduced dimensionality
2022-11-03 18:06:44,386 - BERTopic - Clustered reduced embeddings


     Topic  Count                                               Name
0       -1  12075                      -1_años_puerto_personas_parte
1        0    479                         0_ucrania_rusia_ruso_putin
2        1    354                    1_kast_boric_candidato_gobierno
3        2    229        2_fuente_fuente cooperativa_gobierno_retiro
4        3    201  3_teléfono 652534903652534904_pérez domicilio_...
..     ...    ...                                                ...
540    539      8  539_brigada_centro veterinario municipal_veter...
541    540      8               540_colo_jugadores_plantel_colo colo
542    541      8         541_agua potable_potable_agua_colonia tres
543    542      8  542_fndr_organizaciones_fondo concursable fndr...
544    543      8                  543_senadores_votos_aportes_mesas

[545 rows x 3 columns]
n_neighbors:  10
n_components:  4
min_cluster_size:  10


2022-11-03 18:12:19,290 - BERTopic - Transformed documents to Embeddings
2022-11-03 18:13:08,805 - BERTopic - Reduced dimensionality
2022-11-03 18:13:28,201 - BERTopic - Clustered reduced embeddings


     Topic  Count                                               Name
0       -1  10421                      -1_años_personas_puerto_parte
1        0   3095                         0_colo_partido_equipo_club
2        1    479                         1_ucrania_rusia_ruso_putin
3        2    428                    2_boric_kast_candidato_gobierno
4        3    229    3_fuente_gobierno_fuente cooperativa_presidente
..     ...    ...                                                ...
404    403     10  403_escuelas especiales_escuelas_especiales_su...
405    404     10  404_vehicular_semáforos_restricción vehicular_...
406    405     10      405_imputados_operativo_legalidad_incautación
407    406     10  406_derecho aprovechamiento_aprovechamiento_ag...
408    407     10  407_turismo_agroecología_sustentable_reactivación

[409 rows x 3 columns]
n_neighbors:  10
n_components:  4
min_cluster_size:  15


...

     Topic  Count                                            Name
0       -1  12600                   -1_personas_años_puerto_parte
1        0   3081                      0_colo_partido_equipo_club
2        1   1198                    1_película_disco_the_canción
3        2    497                   2_casos_dosis_contagios_salud
4        3    454                      3_ucrania_rusia_ruso_putin
..     ...    ...                                             ...
272    271      8                271_quilineja_raquel_telar_fibra
273    272      8  272_microbasurales_contenedores_cámaras_basura
274    273      8              273_álvarez_examen_jenny_vacunarse
275    274      8               274_eei_espacial_astronautas_nasa
276    275      8                  275_robos_banda_robo_detenidos

[277 rows x 3 columns]
n_neighbors:  50
n_components:  6
min_cluster_size:  10


2022-11-03 21:06:33,664 - BERTopic - Transformed documents to Embeddings
2022-11-03 21:07:09,507 - BERTopic - Reduced dimensionality
2022-11-03 21:07:10,827 - BERTopic - Clustered reduced embeddings


     Topic  Count                                               Name
0       -1  12799                      -1_personas_años_puerto_parte
1        0   3081                         0_colo_partido_equipo_club
2        1   1198                       1_película_disco_the_canción
3        2    497                      2_casos_dosis_contagios_salud
4        3    454                         3_ucrania_rusia_ruso_putin
..     ...    ...                                                ...
222    221     10        221_carabineros_conductor_personal_policial
223    222     10        222_maquinarias_ataque_incendiario_máquinas
224    223     10               223_hospital_niño_frutillar_paciente
225    224     10              224_cristo_hogar_obreros_hogar cristo
226    225     10  225_incendios_conaf_incendios forestales_fores...

[227 rows x 3 columns]
n_neighbors:  50
n_components:  6
min_cluster_size:  15


2022-11-03 21:11:16,288 - BERTopic - Transformed documents to Embeddings
2022-11-03 21:11:55,721 - BERTopic - Reduced dimensionality
2022-11-03 21:11:56,946 - BERTopic - Clustered reduced embeddings


   Topic  Count                                               Name
0     -1     16                     -1_aceite_ingredientes_sal_pcr
1      0  22523                      0_años_personas_región_puerto
2      1   3081                         1_equipo_partido_colo_tras
3      2    217                 2_gobierno_presidente_fuente_chile
4      3    201  3_castro_carabineros133 investigaciones134_bla...
5      4     37                    4_sal_aceite_aceite oliva_oliva
6      5     27                     5_ingredientes_taza_aceite_sal
7      6     25  6_derecho aprovechamiento_aprovechamiento_agua...
